In [ ]:
import os
import gc
import feather
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize

'''
from: 
https://www.kaggle.com/sudalairajkumar/simple-exploration-baseline-ga-customer-revenue
'''
def load_df(csv_path='../input/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

In [ ]:
%%time
train = load_df()
test = load_df("../input/test.csv")

In [ ]:
%%time
feather.write_dataframe(train, 'train.ftr')
feather.write_dataframe(test, 'test.ftr')

In [ ]:
del train, test; gc.collect()

In [ ]:
%%time
train = feather.read_dataframe('train.ftr')
test = feather.read_dataframe('test.ftr')

In [ ]:
train.head()